In [1]:
import pandas as pd
import numpy as np
import math
from pandas.tseries.offsets import DateOffset
from datetime import timedelta
import time
from datetime import datetime, timedelta

In [2]:
df=pd.read_csv("Grocery_Inventory_and_Sales_Dataset.csv")
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/29/2024,5/8/2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,6/10/2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/11/2024,10/5/2024,15068 Scoville Court,62,25,Backordered


In [3]:
df.isnull().sum()

Product_ID                 0
Product_Name               0
Catagory                   1
Supplier_ID                0
Supplier_Name              0
Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Date_Received              0
Last_Order_Date            0
Expiration_Date            0
Warehouse_Location         0
Sales_Volume               0
Inventory_Turnover_Rate    0
Status                     0
dtype: int64

In [4]:
df=df.dropna()

In [5]:
df.isnull().sum()

Product_ID                 0
Product_Name               0
Catagory                   0
Supplier_ID                0
Supplier_Name              0
Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Date_Received              0
Last_Order_Date            0
Expiration_Date            0
Warehouse_Location         0
Sales_Volume               0
Inventory_Turnover_Rate    0
Status                     0
dtype: int64

In [6]:
#del unwanted col
df = df.drop(['Last_Order_Date', 'Warehouse_Location','Sales_Volume','Inventory_Turnover_Rate','Status' ], axis=1)
len(df)

989

In [7]:
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,9/19/2024
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/8/2024
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,9/22/2024
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,4/17/2024
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/5/2024


In [8]:
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'], format='%m/%d/%Y',errors='coerce')
df['Date_Received'] = pd.to_datetime(df['Date_Received'], format='%m/%d/%Y',errors='coerce')

In [11]:
# Calculate shelf life in days
df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days

# Swap values of exp and rec dates only where Shelf_Life_(Days) < 0
mask = df['Shelf_Life_(Days)'] < 0
# Use .loc to safely swap the values conditionally
df.loc[mask, ['Date_Received', 'Expiration_Date']] = df.loc[mask, ['Expiration_Date', 'Date_Received']].values
df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days
df['Shelf_Life_(Years)'] = (df['Shelf_Life_(Days)']/365).round(2)

max_years = math.ceil(df['Shelf_Life_(Years)'].max())
bins = list(range(0, max_years + 1))
labels = [f'{i}-{i+1} years' for i in range(0, max_years)]
df['Shelf_Life_Bin'] = pd.cut(df['Shelf_Life_(Years)'], bins=bins, labels=labels, right=False)
shelf_life_distribution = df['Shelf_Life_Bin'].value_counts().sort_index()

df.reset_index(drop=True,inplace=True)

# View a few rows
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2024-08-16,2024-09-19,34,0.09,0-1 years
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-05-08,2024-11-01,177,0.48,0-1 years
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2024-08-03,2024-09-22,50,0.14,0-1 years
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-04-17,2024-12-08,235,0.64,0-1 years
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2024-07-03,2024-10-05,94,0.26,0-1 years


In [12]:
condition = df['Date_Received'] < pd.Timestamp('2024-11-01')

# Apply 7-month shift only to matching rows
df.loc[condition, 'Date_Received'] = df.loc[condition, 'Date_Received'] + pd.DateOffset(months=7)
df.loc[condition, 'Expiration_Date'] = df.loc[condition, 'Expiration_Date'] + pd.DateOffset(months=7)

# View result
df[['Date_Received', 'Expiration_Date']].head()

,Date_Received,Expiration_Date
0,2025-03-16,2025-04-19
1,2024-12-08,2025-06-01
2,2025-03-03,2025-04-22
3,2024-11-17,2025-07-08
4,2025-02-03,2025-05-05


In [13]:
print(df['Date_Received'].max())
print(df['Date_Received'].min())

2025-05-31 00:00:00
2024-09-25 00:00:00


In [14]:
#making col for donation and discount dates in df

In [15]:
# Apply conditional logic for discount days:
# 20% of shelf life if bin is '0-1 years', otherwise 10%
discount_fraction = np.where(df['Shelf_Life_Bin'] == '0-1 years', 0.2, 0.1)
df['Discount_Date'] = df['Expiration_Date'] - pd.to_timedelta(df['Shelf_Life_(Days)'] * discount_fraction, unit='D')

print(df.head())

    Product_ID     Product_Name             Catagory  Supplier_ID  \
0  29-205-1132       Sushi Rice      Grains & Pulses  38-037-1699   
1  40-681-9981   Arabica Coffee            Beverages  54-470-2479   
2  06-955-3428       Black Rice      Grains & Pulses  54-031-2945   
3  71-594-6552  Long Grain Rice      Grains & Pulses  63-492-7603   
4  57-437-1828             Plum  Fruits & Vegetables  54-226-4308   

  Supplier_Name  Stock_Quantity  Reorder_Level  Reorder_Quantity Unit_Price  \
0     Jaxnation              22             72                70     $4.50    
1       Feedmix              45             77                 2    $20.00    
2        Vinder              30             38                83     $6.00    
3    Brightbean              12             59                62     $1.50    
4    Topicstorm              37             30                74     $4.00    

  Date_Received Expiration_Date  Shelf_Life_(Days)  Shelf_Life_(Years)  \
0    2025-03-16      2025-04-19     

In [16]:
# Set donation fraction based on shelf life bin
donation_fraction = np.where(df['Shelf_Life_Bin'] == '0-1 years', 0.1, 0.05)
df['Donation_Date'] = df['Expiration_Date'] - pd.to_timedelta(df['Shelf_Life_(Days)'] * donation_fraction, unit='D')

df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Discount_Date,Donation_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19,34,0.09,0-1 years,2025-04-12 04:48:00,2025-04-15 14:24:00
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-12-08,2025-06-01,177,0.48,0-1 years,2025-04-26 14:24:00,2025-05-14 07:12:00
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22,50,0.14,0-1 years,2025-04-12 00:00:00,2025-04-17 00:00:00
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-11-17,2025-07-08,235,0.64,0-1 years,2025-05-22 00:00:00,2025-06-14 12:00:00
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05,94,0.26,0-1 years,2025-04-16 04:48:00,2025-04-25 14:24:00


In [31]:
#making a new csv
df.to_csv('grocery_df.csv', index=False)